# CUDA.jl

La biblioteca `CUDA.jl` te permite programar GPU's de NVIDIA usando el lenguaje de programación Julia. Este paquete permite la programación a varios **niveles de abstracción**, desde el uso de _arreglos_, hasta la escritura de _kernels_ usando la API de CUDA en un nivel más bajo.

## Primeros Pasos

Lo primero es instalar el paquete usando `Pkg.add("CUDA")`. Lo único que necesita es teer instalado un controlador de NVIDIA. El toolkit de CUDA se descargará automáicamente al usar por primera vez la librería. Posteriormente, solo queda utilizarlo.

In [5]:
using BenchmarkTools, Test

In [1]:
using CUDA

La primer gran herramienta con la que cuenta `CUDA.jl` es el tipo de dato `CuArray`. Este tipo de dato es igual a un arreglo normal con la peculiaridad de que se almacena en la memoria del GPU, así como que las operaciones realizadas con éstos se harán de manera automática en el GPU. 

Esta implementación es de tan alto nivel que no es necesario utilizar los arreglos dentro de un kernel específico para el GPU, si no que puedes utilizarlo tal y como viene.

In [8]:
N = 2^20
x_d = CUDA.fill(1.0f0, N)
y_d = CUDA.fill(2.0f0, N)

1048576-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 ⋮
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0

Por ejemplo, para sumar ambos arreglos, podemos usar las características del Julia, en este caso, **broadcasting**.

In [9]:
y_d .+= x_d

@test all(Array(y_d) .== 3.0f0)

Test Passed

## Escribiendo un Kernel

Aunque la implementación de alto nivel del uso de arreglos de CUDA fue bastante sencilla, seguimos sin saber qué es lo que está pasando. Si quisiéramos hacer un primer intento para implementar la misma operación en un kernel, haríamos lo siguiente.

In [10]:
function gpu_add1!(y, x)
    for i = 1:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

fill!(y_d, 2)

@cuda gpu_add1!(y_d,x_d)
@test all(Array(y_d) .== 3.0f0)

Test Passed

### Algunas cosas ha resaltar en el programa anterior

Un **kernel** es una función que se ejecuta dentro del GPU. Para programar un kernel en Julia, se escribe una función común. Posteriormente hablaremos sobre API's que puedes usar dentro del kernel para el manejo de datos dentro del GPU. 

Algo importante a la hora de escribir un kernel es que la función **no debe regresar nada**. Es por eso que esta función al final tiene la sentencia `return nothing`, aunque podría ser simplemente un `return`.

Otra cosa a destacar es la forma en que invocamos a la función. Para hacerlo, es necesario utilizar la macro `@cuda` y llamar a la función. Dentro de esta macro se agregan los parámetros de la llamada al kernel, que veremos más adelante.

Sin embargo, si comparamos los tiempos de ejecución del kernel que escribimos con la implementación en alto nivel, veremos que algo está pasando.

In [14]:
function add_broadcast!(y,x)
    CUDA.@sync y .+= x #para sincronizar la ejecución de la función y que se espere a que termine
    return
end

@benchmark add_broadcast!($y_d, $x_d)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  66.903 μs … 185.503 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     68.445 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   68.919 μs ±   2.487 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▃▅▇█▇▆▄▃▁                                               
  ▁▁▂▃▆██████████▇▅▅▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  66.9 μs         Histogram: frequency by time         75.3 μs <

 Memory estimate: 592 bytes, allocs estimate: 11.

In [15]:
function bench_gpu1!(y,x)
    CUDA.@sync begin
        @cuda gpu_add1!(y,x)
    end
end

@benchmark bench_gpu1!($y_d,$x_d)

BenchmarkTools.Trial: 91 samples with 1 evaluation.
 Range (min … max):  54.790 ms … 55.083 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     54.957 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   54.951 ms ± 41.173 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                         ▂ ▁ ▂█▅▁▇             
  ▃▁▁▃▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▃▁▁▁▅▃▁▃▅▅█▆█████████▅▃▁▃▃▅▃▅▃ ▁
  54.8 ms         Histogram: frequency by time          55 ms <

 Memory estimate: 3.25 KiB, allocs estimate: 52.

Como podemos observar, el tiempo de ejecución de nuestro kernel es **significativamente** más tardado que la implementación en alto nivel. Esto es debido a que no estamos aprovechando las capacidades de ejecución en paralelo del GPU. En este caso, nuestro kernel se está ejecutando a un solo hilo dentro del GPU (que es considerablemente más lento que la ejecución serial en CPU). Es por eso que es necesario escribir un kernel **en paralelo**.

## Escribiendo un kernel en paralelo

Para paralelizarlo, es necesario asignar diferentes tareas a diferentes hilos. Para facilitar esto, cada hilo en el GPU tiene acceso a diferentes variables que los identifican de manera única, por ejemplo, su **id de hilo**.

Los hilos de CUDA cuentan con las variables `threadIdx` y `blockDim` que representan el índice del hilo y el número de bloques en ejecución (véase conceptos base de CUDA). Cada variable tiene 3 campos `x`, `y` y `z`. Podemos utilizar lo anterior para escribir el siguiente kernel.

In [18]:
function gpu_add2!(y,x)
    index  = threadIdx().x
    stride = blockDim().x
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

fill!(y_d, 2)
@cuda threads=256 gpu_add2!(y_d,x_d)
@test all(Array(y_d) .== 3.0f0)

Test Passed

Nótese que al invocar el kernel, agregamos la opción `threads=256`, lo que divide el trabajo entre 256 hilos de manera lineal. Ahora veamos su tiempo de ejecución.

In [19]:
function bench_gpu2!(y,x)
    CUDA.@sync begin
        @cuda threads=256 gpu_add2!(y,x)
    end
end

@benchmark bench_gpu2!($y_d, $x_d)


BenchmarkTools.Trial: 2803 samples with 1 evaluation.
 Range (min … max):  1.632 ms …   6.370 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.744 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.767 ms ± 112.153 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▂▂▃▅▅█▇▇█▄▆▄▂           ▁▅▁                         
  ▁▁▁▂▂▄▅▇▇█████████████▇▆▅▄▃▃▃▄▇█████▇▆▇▇▄▄▃▃▃▂▂▃▂▂▂▂▂▂▂▁▁▁▁ ▄
  1.63 ms         Histogram: frequency by time        1.97 ms <

 Memory estimate: 3.11 KiB, allocs estimate: 49.

Aun que ya mejoró bastante el tiempo de ejecución, aún seguimos por debajo de la implementación de alto nivel. Para alcanzarlo, es necesario paralelizar un poco más.

Las GPU tienen un número limitado de hilos que pueden correr en un solo _streaming multiprocessor_ (SM), pero también tienen varios SM. Para utilizarlos todos, es necesario correr el kernel en múltiples _bloques_. Dividiremos el trabajo de la siguiente manera:


<img src="https://cuda.juliagpu.org/stable/tutorials/intro1.png" width=1000>

El diagrama muestra la numeración de los bloques en la biblioteca para C/C++. En Julia, los bloques comienzan con 1 en lugar de 0.


In [20]:
function gpu_add3!(y,x)
    index  = (blockIdx().x -1) * blockDim().x + threadIdx().x
    stride = gridDim().x * blockDim().x 

    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

numblocks = ceil(Int, N/256)

fill!(y_d,2)
@cuda threads=256 blocks=numblocks gpu_add3!(y_d,x_d)
@test all(Array(y_d) .== 3.0f0)

Test Passed

In [21]:
function bench_gpu3!(y,x)
    numblocks = ceil(Int, length(y)/256)
    CUDA.@sync begin
        @cuda threads=256 blocks=numblocks gpu_add3!(y,x)
    end
end

@benchmark bench_gpu3!($y_d, $x_d)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  62.592 μs … 170.801 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     63.868 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   64.062 μs ±   1.771 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▃▄▅▄▅▅▆█▇▆▄                                              
  ▁▂▃▆█████████████▆▄▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  62.6 μs         Histogram: frequency by time         69.2 μs <

 Memory estimate: 336 bytes, allocs estimate: 5.

Como se puede observar, el tiempo de ejecución mejoró aun más, alcanzando la implementación de alto nivel. Con estos elementos ya podrías construir tus propios kernels y ejecutarlos con el número de hilos y bloques que desees, así como usar todo el potencial de los `CuArray`.

## Lenguaje de Kernel

Vamos a revisar algunas funcionalidades del paquete `CUDA.jl` que permiten escribir kernels todavía más poderosos, acercándose al poder de CUDA.c

### Índices y Dimensiones

- `gridDim()`: Regresa la dimensiones de la malla.

- `blockIdx()`: Regresa el índice del bloque dentro de la malla.
- `blockDim()`: Regresa las dimensiones del bloque.
- `threadIdx()`: Regresa el índice del hilo dentro del bloque.
- `warpsize()`: Regresa el tamaño del warp (conjunto de hilos).
- `laneid()`: Regresa el carril del hilo dentro del warp.

## Tipos de Memorias

### Memoria Compartida

- `CuStaticSharedArray(T::Type, dims)`: Devuelve un arreglo de tipo `T` y dimensiones `dims` que apunta a un pedazo de memoria compartida que se asignó de manera estática. El tipo se debe inferir de manera estática y las dimensiones deben ser constantes; de lo contrario, marcará un error.

- `CuDynamicSharedArray(T::Type, dims, offset::Integer=0)`: Devuelve un arreglo de tipo `T`y dimensiones `dims` que apuntan a un pedazo de memoria compartida asignada de manera dinámica. El tipo debe de inferirse de manera estática, de lo contrario marcará un error. La cantidad de memoria que se asignará de manera dinámica debe de especificarse al lanzar el kernel.


                @cuda threads=n_threads blocks=n_blocks shmem=memSize kernel(d_out, d_in)

### Memoria Texture

- `CuTexture{T,N,P}`: Memoria de tipo texture `N`-dimensional de tipo `T`. Estos objetos no almacenan memoria por sí solas, si no que se pasan como parámetro al kernel, donde interactuará con el tipo `CuDeviceTexture`. (**Atención: API experimental suejta a cambios**).

## Sincronización

- `CUDA.sync_threads()`: Espera a que todos los hilos dentro del bloque hayan llegado al punto, y todos los accesos a memoria global realizados antes de la llamada sean visibles para todos los hilos.

- `CUDA.sync_threads_and(predicado::{Int32, Boolean})`: Similar a `sync_threads()` pero permite evaluar un predicado para todos los hilos dentro del bloque.

Para más información acerca del paquete `CUDA.jl`, puedes revisar la (documentación oficial)[https://cuda.juliagpu.org/stable/] de la biblioteca.

